In [7]:
import os
import wandb
# 381b331eeb89239808cb03e0163f2196237c7186
import numpy as np 
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve

import torch
from torch import nn, optim
from torchvision import models
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2

wandb.login()

wandb: Currently logged in as: buinguyenkhai24 (buinguyenkhai24-hust). Use `wandb login --relogin` to force relogin


True

In [8]:
class CheXpertDataset(Dataset):
    '''
    Custom dataset for CheXpert. Returns a tuple of (PIL.Image, torch.Tensor (float32)).
    Args:
        data (pd.Dataframe): dataset with image path as indexes and columns as labels, all values are 0 (negative) or 1 (postivive)
        root_dir (str): root directory of dataset folder
        mode ('train', 'val'): mode for different augmentation
        transforms (albumentations): augmentation techniques to use
    '''
    def __init__(self, data, root_dir, mode='train', transforms=None):
        self.data = data.to_numpy()
        self.labels = torch.tensor(data.values.astype(np.float32))
        self.root_dir = root_dir
        self.img_paths = [os.path.join(root_dir, img_path) for img_path in data.index]
        self.transform = transforms.get(mode)
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image=np.array(image))['image']

        return (image, label)
    
def get_weighted_sampler(data):
    '''
    Custom Sampler for weighted sampling to deal with unbalanced labels. The class weights are the inverse of the count of positives of a label.
        Args:
            data (pd.Dataframe): dataset with image path as indexes and columns as labels, all values are 0 (negative) or 1 (postivive)
            batch_size (int): number of indices to use at a time 
    '''
    class_weights = (1/data.sum()).values
    weights = data.dot(class_weights)
    weighted_sampler = WeightedRandomSampler(torch.tensor(weights.values, dtype=torch.float), len(weights), replacement=True)
    return weighted_sampler
'''
Augmentations:
    - Scale 5% with p = 50%
    - Rotate 20° OR shear 5 pixels with p = 50%
    - Translate 5% with p = 50%
    - Resize to 224x224
    - Normalize with mean = 0.506 and std = 0.287, more details in data_preprocessing.ipynb
    - Convert to torch.Tensor
'''
transform = {
    'train': A.Compose([
        A.Affine(scale=(0.95, 1.05), p=0.5),
        A.OneOf([A.Affine(rotate=(-20, 20), p=0.5), A.Affine(shear=(-5, 5), p=0.5)], p=0.5),
        A.Affine(translate_percent=(-0.05, 0.05), p=0.5),
        A.Resize(224, 224),
        A.Normalize([0.506, 0.506, 0.506], [0.287, 0.287, 0.287]),
        ToTensorV2()
    ]),
    'val': A.Compose([
        A.Resize(224, 224),
        A.Normalize([0.506, 0.506, 0.506], [0.287, 0.287, 0.287]),
        ToTensorV2()
    ]),
}

In [9]:
test = pd.read_csv('/kaggle/input/chexpertclean/u1_test.csv',index_col=0)
train = pd.read_csv('/kaggle/input/chexpertclean/u1_train.csv', index_col=0)
val = pd.read_csv('/kaggle/input/chexpertclean/u1_val.csv', index_col=0)

In [10]:
train.index =  train.index.str.replace('CheXpert-v1.0-small', 'chexpert')
test.index = test.index.str.replace('CheXpert-v1.0-small', 'chexpert')
val.index = val.index.str.replace('CheXpert-v1.0-small', 'chexpert')

Confirm no data leakage

In [11]:
print(np.intersect1d(train.index.to_numpy(), test.index.to_numpy()))
print(np.intersect1d(val.index.to_numpy(), test.index.to_numpy()))
print(np.intersect1d(val.index.to_numpy(), train.index.to_numpy()))

[]
[]
[]


Configuration for Bayes Hyperparameter Tuning using Wandb

In [12]:
sweep_config = {
    "method": "bayes",
    "metric": {
        "name": "val_mAUC",
        "goal": "maximize"
    },
    "parameters": {
        "learning_rate": {
            "values": [1e-3, 1e-4, 1e-5]
        },
        "weight_decay": {
            "values": [1e-3, 1e-4, 1e-5]
        },
        "patience": {
            "values": [0, 1, 5, 10]
        },
        "drop_rate": {
            "values": [0, 0.1, 0.25, 0.5]
        },
        "val_patience": {
            "values": [5, 10]
        }
    }
}
sweep_id = wandb.sweep(sweep_config, project="deep_learning")

Create sweep with ID: o9oj3pgz
Sweep URL: https://wandb.ai/buinguyenkhai24-hust/deep_learning/sweeps/o9oj3pgz


In [13]:
#model = models.densenet121(pretrained=True)
#for param in model.parameters():
#    param.requires_grad = False
#model.classifier = nn.Linear(in_features=model.classifier.in_features, out_features=14)

In [14]:
batch_size = 32

train_dataset = CheXpertDataset(
    data=train,
    root_dir='/kaggle/input/', 
    mode='train',
    transforms = transform
    )

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    sampler = get_weighted_sampler(train),
    num_workers = 4,
    pin_memory=True
    )

val_dataset = CheXpertDataset(
    val, '/kaggle/input/',
    mode='val',
    transforms = transform)

val_loader = DataLoader(
    val_dataset,
    batch_size = batch_size,
    num_workers = 4,
    pin_memory=True)

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def train(config=None):
    with wandb.init(config=config):
        config = wandb.config
        
        model = models.densenet121(num_classes=14, drop_rate=config.drop_rate).to(device)
        loss_function = nn.BCEWithLogitsLoss()
        optimizer = optim.AdamW(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=config.patience)

        best_val_mAUC = 0
        epochs_without_improvement = 0
        max_epochs = 100
        val_patience = config.val_patience
        for epoch in range(max_epochs):
            
            model.train()
            epoch_loss = 0.0
            for inputs, labels in tqdm((train_loader), desc=f"Epoch {epoch+1}/{max_epochs}", unit="batch"):
                # Compute prediction and loss
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = loss_function(outputs, labels)
        
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
        
                # Accumulate loss
                epoch_loss += loss.item()
        
            # Compute and print average loss for the epoch
            avg_loss = epoch_loss / len(train_loader)
            scheduler.step(avg_loss)
            print(f"Epoch {epoch+1}/{max_epochs}, Loss: {avg_loss:.4f}")
        
            model.eval()
            all_labels = []
            all_preds = []
        
            with torch.no_grad():
                for X, y in tqdm(val_loader, desc="Testing", unit="batch"):
                    inputs, labels = inputs.to(device), labels.to(device)
            
                    # Get the logits from the model
                    outputs = model(inputs)
            
                    # Apply sigmoid to the outputs to get probabilities
                    outputs = torch.sigmoid(outputs)
            
                    # Collect the true labels and predictions
                    all_labels.append(labels.cpu().numpy())  # Convert to numpy and store
                    all_preds.append(outputs.cpu().numpy())  # Convert to numpy and store
            
            # Convert the list of arrays into a single numpy array for true labels and predictions
            all_labels = np.concatenate(all_labels, axis=0)
            all_preds = np.concatenate(all_preds, axis=0)
            
            # Compute AUROC for multi-label classification
            auroc_scores = []
            for i in range(all_labels.shape[1]):  # Loop through each label
                try:
                    auroc = roc_auc_score(all_labels[:, i], all_preds[:, i])
                    auroc_scores.append(auroc)
                except ValueError:
                    pass
        
            val_mAUC = np.mean(auroc_scores)
            print(f'Mean AUROC: {val_mAUC:.4f}')
        
            wandb.log({"epoch": epoch, "train_loss": avg_loss, "val_mAUC": val_mAUC})
            
            if val_mAUC > best_val_mAUC:
                best_val_mAUC = val_mAUC
                torch.save(model.state_dict(), "DenseNetScratch-U1.pth")
                epochs_without_improvement = 0
            else:
                epochs_without_improvement += 1
                
            # Early stopping
            if epochs_without_improvement >= val_patience:
                print("Early stopping triggered!")
                break


In [ ]:
wandb.agent(sweep_id, train, count=10)

wandb: Agent Starting Run: n9pahrq1 with config:
wandb: 	drop_rate: 0.1
wandb: 	learning_rate: 0.0001
wandb: 	patience: 1
wandb: 	val_patience: 5
wandb: 	weight_decay: 1e-05


Epoch 1/100:   4%|▎         | 171/4797 [00:34<13:47,  5.59batch/s]

In [ ]:
# Function to calculate and plot AUROC for multi-label classification
def plot_auroc(model, dataloader, num_classes, device):
    model.eval()
    
    true_labels = []
    predicted_probs = []
    
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Get raw model output (logits)
            outputs = model(inputs)
            
            # Apply sigmoid to get probabilities for each label
            probs = torch.sigmoid(outputs)
            
            true_labels.append(labels.cpu().numpy())
            predicted_probs.append(probs.cpu().numpy())
    
    # Convert to numpy arrays
    true_labels = np.vstack(true_labels)
    predicted_probs = np.vstack(predicted_probs)

    # Calculate AUROC for each label
    fpr = {}
    tpr = {}
    roc_auc = {}

    # Iterate over each label and compute the ROC curve and AUROC
    for i in range(num_classes):
        fpr[i], tpr[i], _ = roc_curve(true_labels[:, i], predicted_probs[:, i])
        roc_auc[i] = roc_auc_score(true_labels[:, i], predicted_probs[:, i])

    # Plot the ROC curve for each label
    plt.figure(figsize=(10, 8))
    for i in range(num_classes):
        plt.plot(fpr[i], tpr[i], label=f'Class {i+1} (AUROC = {roc_auc[i]:.2f})')
    
    # Plot the diagonal (random guess line)
    plt.plot([0, 1], [0, 1], 'k--', label='Random guess (AUROC = 0.5)')
    
    # Add labels and legend
    plt.title('Multi-label (ROC) Curve')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc='lower right')
    plt.show()
    
    # Return mean AUROC
    mean_auroc = np.mean(list(roc_auc.values()))
    print(f'Mean AUROC across all classes: {mean_auroc:.2f}')
    return mean_auroc

#model.to(device)
#plot_auroc(model, val_loader, num_classes=14, device='cuda')